### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [75]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head(10)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [73]:
overall_pass_rate = (student_data["math_score"].mean()+student_data["reading_score"].mean())/2
total_stu_above_70_math = student_data[student_data["math_score"]>70]["Student ID"].count()
total_stu_above_70_read = student_data[student_data["reading_score"]>70]["Student ID"].count()
total_stu = student_data["Student ID"].count()

district_summary = pd.DataFrame({
                "Total Schools":[school_data["School ID"].nunique()],
                "Total Students":["{:,}".format(school_data["size"].sum())],
                "Total Budget":['${:,}'.format(school_data["budget"].sum())],
                "Avg Math Score":['{:.2%}'.format(student_data["math_score"].mean()/100)],
                "Avg Read Score":['{:.2%}'.format(student_data["reading_score"].mean()/100)],
                "Overall Pass Rate":['{:.2%}'.format(overall_pass_rate/100)],
                "Math above 70":['{:.2%}'.format(total_stu_above_70_math/total_stu)],
                "Reading above 70":['{:.2%}'.format(total_stu_above_70_read/total_stu)]})
district_summary


,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Read Score,Overall Pass Rate,Math above 70,Reading above 70
0,15,"39,170","$24,649,428",78.99%,81.88%,80.43%,72.39%,82.97%


In [141]:
#write school details into dataframe
school_summary = pd.DataFrame({"school_name":school_data["school_name"], 
                   "school_type":school_data["type"],
                   "total_students":school_data["size"],
                   "total_budget":school_data["budget"], 
                    "budget_per_student":school_data["budget"]/school_data["size"]})

#group student data by school name
student_summary_gr = student_data.groupby("school_name")

#get counts students passing math by school
student_pass_math = pd.DataFrame(student_data[student_data["math_score"].astype(float)>=70].groupby("school_name")["Student ID"].count()*100/student_summary_gr["Student ID"].count())
student_pass_math.columns = ["passing_math"]

#get counts students passing math by school
student_pass_read = pd.DataFrame(student_data[student_data["reading_score"].astype(float)>=70].groupby("school_name")["Student ID"].count()*100/student_summary_gr["Student ID"].count())
student_pass_read.columns = ["passing_reading"]

#add student avg math score by school to dataframe 
student_summary = pd.DataFrame(student_summary_gr["math_score"].mean()) 

#create another dataframe with avg read score and merge to the above on school name
student_summary = pd.merge(student_summary, pd.DataFrame(student_summary_gr["reading_score"].mean()), on="school_name")

#add passing math and passing reading 
student_summary = pd.merge(student_summary, student_pass_math, on="school_name")
student_summary = pd.merge(student_summary, student_pass_read, on="school_name")

#add overall passing rate
student_summary["overall_passing_rate"] = (student_summary["passing_math"].astype(float)+student_summary["passing_reading"].astype(float))/2


#join with school data
school_summary = pd.merge(school_summary, student_summary, on="school_name")
school_summary["passing_math"] = school_summary["passing_math"].astype(float)

#format data for output
school_summary_format = school_summary.copy()
school_summary_format["passing_math"] = (school_summary_format["passing_math"]/100).map('{:.2%}'.format)
school_summary_format["passing_reading"] = (school_summary_format["passing_reading"]/100).map('{:.2%}'.format)
school_summary_format["overall_passing_rate"] = (school_summary_format["overall_passing_rate"]/100).map('{:.2%}'.format)
school_summary_format["math_score"] = (school_summary_format["math_score"]/100).map('{:.2%}'.format)
school_summary_format["reading_score"] = (school_summary_format["reading_score"]/100).map('{:.2%}'.format)
school_summary_format["total_budget"] = school_summary_format["total_budget"].map('${:,}'.format)
school_summary_format["budget_per_student"] = school_summary_format["budget_per_student"].map('${:,}'.format)

#update column names
school_summary_format.columns = ["School Name", "School Type", "Total Students", "Total School Budget", 
                         "Per Student Budget", "Average Math Score", "Average Reading Score",
                        "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [116]:
school_summary_format.sort_values(by=["% Overall Passing Rate"],ascending=False).head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
6,Cabrera High School,Charter,1858,"$1,081,356",$582.0,83.06%,83.98%,94.13%,97.04%,95.59%
14,Thomas High School,Charter,1635,"$1,043,130",$638.0,83.42%,83.85%,93.27%,97.31%,95.29%
4,Griffin High School,Charter,1468,"$917,500",$625.0,83.35%,83.82%,93.39%,97.14%,95.27%
9,Pena High School,Charter,962,"$585,858",$609.0,83.84%,84.04%,94.59%,95.95%,95.27%
5,Wilson High School,Charter,2283,"$1,319,574",$578.0,83.27%,83.99%,93.87%,96.54%,95.20%


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [117]:
school_summary_format.sort_values(by=["% Overall Passing Rate"]).head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
11,Rodriguez High School,District,3999,"$2,547,363",$637.0,76.84%,80.74%,66.37%,80.22%,73.29%
1,Figueroa High School,District,2949,"$1,884,411",$639.0,76.71%,81.16%,65.99%,80.74%,73.36%
0,Huang High School,District,2917,"$1,910,635",$655.0,76.63%,81.18%,65.68%,81.32%,73.50%
12,Johnson High School,District,4761,"$3,094,650",$650.0,77.07%,80.97%,66.06%,81.22%,73.64%
13,Ford High School,District,2739,"$1,763,916",$644.0,77.10%,80.75%,68.31%,79.30%,73.80%


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [118]:
math_grade_9 = pd.DataFrame(student_data[student_data["grade"]=='9th'].groupby("school_name")["math_score"].mean())
math_grade_10 = pd.DataFrame(student_data[student_data["grade"]=='10th'].groupby("school_name")["math_score"].mean())
math_grade_11 = pd.DataFrame(student_data[student_data["grade"]=='11th'].groupby("school_name")["math_score"].mean())
math_grade_12 = pd.DataFrame(student_data[student_data["grade"]=='12th'].groupby("school_name")["math_score"].mean())

math_scores = pd.merge(math_grade_9, math_grade_10, on="school_name")
math_scores = pd.merge(math_scores, math_grade_11, on="school_name")
math_scores = pd.merge(math_scores, math_grade_12, on="school_name")

math_scores.columns = ["9th Avg", "10th Avg", "11th Avg", "12th Avg"]

math_scores

,9th Avg,10th Avg,11th Avg,12th Avg
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [119]:
read_grade_9 = pd.DataFrame(student_data[student_data["grade"]=='9th'].groupby("school_name")["reading_score"].mean())
read_grade_10 = pd.DataFrame(student_data[student_data["grade"]=='10th'].groupby("school_name")["reading_score"].mean())
read_grade_11 = pd.DataFrame(student_data[student_data["grade"]=='11th'].groupby("school_name")["reading_score"].mean())
read_grade_12 = pd.DataFrame(student_data[student_data["grade"]=='12th'].groupby("school_name")["reading_score"].mean())

read_scores = pd.merge(read_grade_9, read_grade_10, on="school_name")
read_scores = pd.merge(read_scores, read_grade_11, on="school_name")
read_scores = pd.merge(read_scores, read_grade_12, on="school_name")

read_scores.columns = ["9th Avg", "10th Avg", "11th Avg", "12th Avg"]

read_scores

,9th Avg,10th Avg,11th Avg,12th Avg
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [137]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#cut the dataframe into spending bins
school_summary["Spending Ranges (Per Student)"] = pd.cut(school_summary["budget_per_student"], spending_bins, labels=group_names)

#create grouping based on the new spending size
school_summary_gr = school_summary.groupby("Spending Ranges (Per Student)")

#create new df from the group object using mean function
school_summary_by_budget = pd.DataFrame(school_summary_gr["math_score", "reading_score", "passing_math", "passing_reading", "overall_passing_rate"].mean())
school_summary_by_budget.columns = ["Avg Math Score", "Avg Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]
school_summary_by_budget

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [138]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#cut the data into bins
school_summary["School Size"] = pd.cut(school_summary["total_students"], size_bins, labels=group_names)

#group by new bin size column
school_summary_gr = school_summary.groupby("School Size")

#create new df from the group using mean()
school_summary_by_size = pd.DataFrame(school_summary_gr["math_score", "reading_score", "passing_math", "passing_reading", "overall_passing_rate"].mean())
school_summary_by_size.columns = ["Avg Math Score", "Avg Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]
school_summary_by_size

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [140]:
school_summary_gr = school_summary.groupby("school_type")
school_summary_by_type = pd.DataFrame(school_summary_gr["math_score", "reading_score", "passing_math", "passing_reading", "overall_passing_rate"].mean())
school_summary_by_type.columns = ["Avg Math Score", "Avg Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]
school_summary_by_type

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
